In [1]:
import numpy as np
import pandas as pd

# Total Return Swaps

In [2]:
def TRS(n_bonds, coupons, initial_prices, final_prices, fixed_rate, t, pay_total_return):
     """
     Calculate the profit/loss of a total return swap.

     Parameters
     ----------
     n_bonds : array-like
         Number of bonds.
     coupons : array-like
         Coupon rates of the bonds.
     initial_prices : array-like
         Initial prices of the bonds.
     final_prices : array-like
         Final prices of the bonds.
     fixed_rate : array-like
         Fixed rate for the swap.
     t : array-like
         Time period for the swap.
     pay_total_return : bool, optional
         If True, pay total return. Default is True.

     Returns
     -------
     None
         Prints the profit/loss from the total return swap.
     """
     initial_port_value = np.dot(n_bonds, initial_prices)
     final_port_value = np.dot(n_bonds, final_prices)
     coupon_pmt = np.sum(n_bonds * coupons/t * 100)
     total_return = (final_port_value - initial_port_value) + coupon_pmt
     rate = fixed_rate/t * initial_port_value

     return print(f'${(rate - total_return if pay_total_return else total_return - rate)[0]:,.2f}')

In [3]:
n_bonds = np.array([35_000, 45_000, 50_000])
coupons = np.array([0.06, 0.07, 0.08])
initial_prices = np.array([98.75, 101.5, 97.6])
final_prices = np.array([96.4, 97.3, 92.8])
fixed_rate = np.array([0.04])
t = np.array([2])
pay_total_return = True

TRS(n_bonds, coupons, initial_prices, final_prices, fixed_rate, t, pay_total_return)

$306,825.00


# Credit Default Swaps